<table align="left">
    <tr>
        <td style="vertical-align: middle; padding-left: 0px; padding-right: 0px;">
            <a href="https://creativecommons.org/licenses/by/4.0/">
                <img src="https://licensebuttons.net/l/by/4.0/80x15.png" />
            </a>
        </td>
        <td style="vertical-align: middle; padding-left: 5px; padding-right: 0px;">
            <a href="https://opensource.org/licenses/MIT">
                <img src="https://img.shields.io/badge/License-MIT-green.svg" />
            </a>
        </td>
        <td style="vertical-align: middle; padding-left: 15px;">
            &copy; Guillaume Rongier
        </td>
    </tr>
</table>

# Adding subsidence

This notebook starts from the basic example of the [first notebook](1_basic-example.ipynb) to show how to add uplift and subsidence in StratigraPy.

### Imports

Let's first import all the required packages and components:

In [ ]:
import numpy as np
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import cmocean

from landlab.components import FlowDirectorMFD, FlowAccumulator
from landlab.components.flexure import Flexure

from stratigrapy import RasterModelGrid
from stratigrapy.components import SeaLevelCalculator, WaterDrivenRouter

## 1. Simple subsidence

We'll use the same case as in the [first notebook](1_basic-example.ipynb), starting with the same simulation time:

In [ ]:
timestep = 100.
runtime = 500000.
n_iterations = int(runtime/timestep)

Then the grid:

In [ ]:
grid = RasterModelGrid((25, 30),
                       xy_spacing=(2500., 2500.),
                       number_of_classes=2,
                       initial_allocation=n_iterations//100 + 100,
                       number_of_layers_to_fuse=100,
                       number_of_top_layers=100,
                       fuse_continuously=True)

In [ ]:
grid.set_closed_boundaries_at_grid_edges(True, True, True, False)

The initial topography:

In [ ]:
elevation = grid.add_zeros('topographic__elevation', at='node', clobber=True)
elevation += 0.003*(grid.y_of_node - 50000.)

The sources of water and sediments:

In [ ]:
idx = np.ravel_multi_index(((23, 23), (14, 15)), grid.shape)
water_influx = grid.add_zeros('water__unit_flux_in', at='node', clobber=True)
water_influx[idx] = 5000. # m/yr

In [ ]:
sediment_influx = grid.add_field('sediment__unit_flux_in',
                                 np.zeros((grid.number_of_nodes, 2)),
                                 clobber=True)
sediment_influx[idx] = [0.7*50000., 0.3*50000.] # m3/yr

The component controlling sea level:

In [ ]:
slc = SeaLevelCalculator(grid, wavelength=[100000., 10000.], amplitude=[25., 2.5])

The component controlling water flow:

In [ ]:
fd = FlowDirectorMFD(grid, partition_method='slope', diagonals=True)

In [ ]:
fa = FlowAccumulator(grid, flow_director=fd)

The component controlling sediment transport:

In [ ]:
wdr = WaterDrivenRouter(grid,
                        transportability_cont=[1e-8, 1e-8],
                        transportability_mar=[4e-10, 2e-10],
                        wave_base=15.,
                        max_erosion_rate_sed=1e-2,
                        max_erosion_rate_br=1e-12,
                        bedrock_composition=[0.7, 0.3],
                        fields_to_track='bathymetric__depth')

Finally, we define a map of uplift rate, here defining a subsidence that increases linearly towards the deeper part of the domain:

In [ ]:
uplift = grid.add_zeros('uplift__rate', at='node', clobber=True)
uplift += -2e-4 + (2e-4 - 1e-8)*(grid.y_of_node - grid.y_of_node.min())/np.ptp(grid.y_of_node)

In [ ]:
grid.imshow('uplift__rate', var_name='Uplift rate', var_units='m/yr',
            grid_units=['m', 'm'], cmap='magma_r')

To add the subsidence during the simulation, we simply update the topography:

In [ ]:
for i in tqdm(range(n_iterations)):
    elevation += uplift*timestep
    slc.run_one_step(timestep)
    fa.run_one_step()
    wdr.run_one_step(timestep)
    grid.stacked_layers.fuse(time=np.mean, bathymetric__depth=np.mean)
grid.stacked_layers.fuse(finalize=True, time=np.mean, bathymetric__depth=np.mean)

Let's visualize the result:

In [ ]:
fig, ax = plt.subplots()

raster_x = grid.x_of_node[grid.core_nodes].reshape(grid.cell_grid_shape)
raster_y = grid.y_of_node[grid.core_nodes].reshape(grid.cell_grid_shape)
raster_z = grid.at_node['topographic__elevation'][grid.core_nodes].reshape(grid.cell_grid_shape)

pc = ax.pcolormesh(raster_x, raster_y, raster_z, cmap=cmocean.cm.topo,
                   norm=mcolors.CenteredNorm(grid.at_grid['sea_level__elevation']))
fig.colorbar(pc, ax=ax, label='Elevation (m)')

ax.set(xlabel='y (m)', ylabel='y (m)', aspect='equal');

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5.4))

# Sediments
pc = grid.plot_layers(ax, 'bathymetric__depth', cmap=cmocean.cm.deep, zorder=2)
fig.colorbar(pc[0], ax=ax, label='Water depth (m)')

raster_y = grid.y_of_node[grid.core_nodes].reshape(grid.cell_grid_shape)[:, 14]
raster_z = grid.at_node['topographic__elevation'][grid.core_nodes].reshape(grid.cell_grid_shape)[:, 14]
# Sea level
fill_sea = ax.fill_between(raster_y, raster_z, grid.at_grid['sea_level__elevation'],
                           color='#c6dbef', zorder=0)
# Bedrock
ymin, ymax = ax.get_ylim()
ax.fill_between(raster_y, raster_z, ymin, color='#d9d9d9', zorder=1)

ax.set(xlabel='y (m)', ylabel='z (m)', ylim=(ymin, ymax));

## 2. Lithospheric flexure

Landlab also has a component for lithospheric flexure that can be used with StratigraPy. Even though it doesn't make total sense, let's keep the same example:

In [ ]:
grid = RasterModelGrid((25, 30),
                       xy_spacing=(2500., 2500.),
                       number_of_classes=2,
                       initial_allocation=n_iterations//100 + 100,
                       number_of_layers_to_fuse=100,
                       number_of_top_layers=100,
                       fuse_continuously=True)
grid.set_closed_boundaries_at_grid_edges(True, True, True, False)

In [ ]:
elevation = grid.add_zeros('topographic__elevation', at='node', clobber=True)
elevation += 0.003*(grid.y_of_node - 50000.)

In [ ]:
idx = np.ravel_multi_index(((23, 23), (14, 15)), grid.shape)
water_influx = grid.add_zeros('water__unit_flux_in', at='node', clobber=True)
water_influx[idx] = 5000. # m/yr
sediment_influx = grid.add_field('sediment__unit_flux_in',
                                 np.zeros((grid.number_of_nodes, 2)),
                                 clobber=True)
sediment_influx[idx] = [0.7*50000., 0.3*50000.] # m3/yr

In [ ]:
slc = SeaLevelCalculator(grid, wavelength=[100000., 10000.], amplitude=[25., 2.5])

In [ ]:
fd = FlowDirectorMFD(grid, partition_method='slope', diagonals=True)
fa = FlowAccumulator(grid, flow_director=fd)

In [ ]:
wdr = WaterDrivenRouter(grid,
                        transportability_cont=[1e-8, 1e-8],
                        transportability_mar=[4e-10, 2e-10],
                        wave_base=15.,
                        max_erosion_rate_sed=1e-2,
                        max_erosion_rate_br=1e-12,
                        bedrock_composition=[0.7, 0.3],
                        fields_to_track='bathymetric__depth')

And setup the Flexure component instead of an uplift rate map as explained [here](https://landlab.readthedocs.io/en/latest/tutorials/flexure/lots_of_loads.html):

In [ ]:
load = grid.add_zeros('lithosphere__overlying_pressure_increment', at='node', clobber=True)

In [ ]:
flex = Flexure(grid, eet=3000., method='flexure')

To compute the flexure, we need to define the sediment density, here assuming that the two classes have the same density:

In [ ]:
density_sed = 2650. # kg/m3

And we need to define arrays to store the total sediment thickness:

In [ ]:
thickness_sed = grid.add_zeros('sediments__thickness', at='node', clobber=True)
thickness_sed[grid.core_nodes] = grid.stacked_layers.thickness
prev_thickness_sed = thickness_sed.copy()

Now we can run the simulation, adding the computation of load change at each timestep and applying that change to the topography:

In [ ]:
for i in tqdm(range(n_iterations)):
    slc.run_one_step(timestep)
    fa.run_one_step()
    wdr.run_one_step(timestep)

    # This looks alright, but Landlab's tutorial doesn't show how to do this step
    thickness_sed[grid.core_nodes] = grid.stacked_layers.thickness
    load[:] = density_sed*flex.gravity*(thickness_sed - prev_thickness_sed)
    flex.update()
    elevation -= grid.at_node['lithosphere_surface__elevation_increment']
    prev_thickness_sed[:] = thickness_sed

    grid.stacked_layers.fuse(time=np.mean, bathymetric__depth=np.mean)
grid.stacked_layers.fuse(finalize=True, time=np.mean, bathymetric__depth=np.mean)

Let's visualize the result:

In [ ]:
fig, ax = plt.subplots()

raster_x = grid.x_of_node[grid.core_nodes].reshape(grid.cell_grid_shape)
raster_y = grid.y_of_node[grid.core_nodes].reshape(grid.cell_grid_shape)
raster_z = grid.at_node['topographic__elevation'][grid.core_nodes].reshape(grid.cell_grid_shape)

pc = ax.pcolormesh(raster_x, raster_y, raster_z, cmap=cmocean.cm.topo,
                   norm=mcolors.CenteredNorm(grid.at_grid['sea_level__elevation']))
fig.colorbar(pc, ax=ax, label='Elevation (m)')

ax.set(xlabel='y (m)', ylabel='y (m)', aspect='equal');

In [ ]:
fig, ax = plt.subplots(figsize=(10, 3.5))

# Sediments
pc = grid.plot_layers(ax, 'bathymetric__depth', cmap=cmocean.cm.deep, zorder=2)
fig.colorbar(pc[0], ax=ax, label='Water depth (m)')

raster_y = grid.y_of_node[grid.core_nodes].reshape(grid.cell_grid_shape)[:, 14]
raster_z = grid.at_node['topographic__elevation'][grid.core_nodes].reshape(grid.cell_grid_shape)[:, 14]
# Sea level
fill_sea = ax.fill_between(raster_y, raster_z, grid.at_grid['sea_level__elevation'],
                           color='#c6dbef', zorder=0)
# Bedrock
ymin, ymax = ax.get_ylim()
ax.fill_between(raster_y, raster_z, ymin, color='#d9d9d9', zorder=1)

ax.set(xlabel='y (m)', ylabel='z (m)', ylim=(ymin, ymax));